In [ ]:
import math
from typing import TypedDict, Literal


In [ ]:
from langgraph.graph import StateGraph, START, END


In [ ]:
class QuadraticState(TypedDict):
    a: int
    b: int
    c: int
    equation: str
    discriminant: float
    first_root: float | None
    second_root: float | None
    answer: str


In [ ]:
def show_equation(state: QuadraticState) -> QuadraticState:
    a = state['a']
    b = state['b']
    if b > 0:
        b = f"+{b}";
    c = state['c']
    if c > 0:
        c = f"+{c}";
    equation = f"{a}x^2{b}x{c}"
    
    return {'equation': equation};


In [ ]:
def find_discriminant(state: QuadraticState):
    a, b, c = state['a'], state['b'], state['c']
    discriminant = (b**2) - 4 * a * c;
    
    return {'discriminant': discriminant}


In [ ]:
def no_roots(state: QuadraticState):
    state['first_root'] = None
    state['second_root'] = None
    state['answer'] = "No roots found"
    
    return state;


In [ ]:
def common_roots(state: QuadraticState):
    a, b = state['a'], state['b']
    state['first_root'] = state['second_root'] = (-b)/(2*a);
    state['answer'] = f"Roots are common, i.e., {state['first_root']}"
    
    return state;


In [ ]:
def real_roots(state: QuadraticState):
    a, b = state['a'], state['b']
    discriminant = state['discriminant'];
    first_root = ((-b) + (math.sqrt(discriminant))) / (2*a)
    second_root = ((-b) - (math.sqrt(discriminant))) / (2*a)
    state['first_root'] = round(first_root, 2)
    state['second_root'] = round(second_root, 2)
    state['answer'] = f'Real roots are {state["first_root"]} and {state["second_root"]}'
    
    return state


In [ ]:
def check_condition(state: QuadraticState) -> Literal['no_roots', 'common_roots', 'real_roots']:
    if state['discriminant'] < 0:
        return "no_roots"
    elif state['discriminant'] == 0:
        return "common_roots"
    else:
        return "real_roots"


In [ ]:
graph = StateGraph(QuadraticState)

graph.add_node('show_equation', show_equation)
graph.add_node('find_discriminant', find_discriminant)
graph.add_node('no_roots', no_roots)
graph.add_node('common_roots', common_roots)
graph.add_node('real_roots', real_roots)

graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'find_discriminant')
graph.add_conditional_edges('find_discriminant', check_condition)
graph.add_edge('no_roots', END)
graph.add_edge('common_roots', END)
graph.add_edge('real_roots', END)

workflow = graph.compile()


In [ ]:
initial_state = {
    'a': 2,
    'b': -8,
    'c': 6,
}

final_state = workflow.invoke(initial_state)


In [ ]:
final_state


{'a': 2,
 'b': -8,
 'c': 6,
 'equation': '2x^2-8x+6',
 'discriminant': 16,
 'first_root': 3.0,
 'second_root': 1.0,
 'answer': 'Real roots are 3.0 and 1.0'}